# deal with converting csv to json when uploading to aws elasticsearch

In [ ]:
import json
import pandas as pd

In [ ]:
json.dumps({'a':1})

In [ ]:
df = pd.read_csv('File_3.csv')

In [ ]:
df.head()

In [ ]:
# { "index" : { "_index": "movies", "_type" : "movie", "_id" : "2" } }
# {"director": "Frankenheimer, John", "genre": ["Drama", "Mystery", "Thriller"], "year": 1962, "actor": ["Lansbury, Angela", "Sinatra, Frank", "Leigh, Janet", "Harvey, Laurence", "Silva, Henry", "Frees, Paul", "Gregory, James", "Bissell, Whit", "McGiver, John", "Parrish, Leslie", "Edwards, James", "Flowers, Bess", "Dhiegh, Khigh", "Payne, Julie", "Kleeb, Helen", "Gray, Joe", "Nalder, Reggie", "Stevens, Bert", "Masters, Michael", "Lowell, Tom"], "title": "The Manchurian Candidate"}
# { "index" : { "_index": "movies", "_type" : "movie", "_id" : "3" } }
# {"director": "Baird, Stuart", "genre": ["Action", "Crime", "Thriller"], "year": 1998, "actor": ["Downey Jr., Robert", "Jones, Tommy Lee", "Snipes, Wesley", "Pantoliano, Joe", "Jacob, Ir\u00e8ne", "Nelligan, Kate", "Roebuck, Daniel", "Malahide, Patrick", "Richardson, LaTanya", "Wood, Tom", "Kosik, Thomas", "Stellate, Nick", "Minkoff, Robert", "Brown, Spitfire", "Foster, Reese", "Spielbauer, Bruce", "Mukherji, Kevin", "Cray, Ed", "Fordham, David", "Jett, Charlie"], "title": "U.S. Marshals"}
# { "index" : { "_index": "movies", "_type" : "movie", "_id" : "4" } }
# {"director": "Ray, Nicholas", "genre": ["Drama", "Romance"], "year": 1955,

In [ ]:
dict_list = []
index_list = []
cnt = 1
for index, row in df.iterrows():
    if row["bestAnswer"] == 1:
        tmp = {}
        tmp["RestaurantID"] = row["RestaurantId"]
        tmp["Cuisine"] = row["Cuisine"]
        tmp["Score"] = row["score"]
        dict_list.append(tmp)
        tmp_index_outer={}
        tmp_index={}
        tmp_index["_index"] = "predictions"
        tmp_index["_type"] = "Prediction"
        tmp_index["_id"] = cnt
        cnt += 1
        tmp_index_outer["index"] = tmp_index
        index_list.append(tmp_index_outer)

In [ ]:
file_cache = []
for i in range(len(dict_list)):
    file_cache.append(json.dumps(index_list[i]))
    file_cache.append(json.dumps(dict_list[i]))

In [ ]:
file_cache

In [ ]:
with open("convertedData.json", "w") as output:
    output.write("\n".join(file_cache))
    output.write("\n")

In [ ]:
# deal with query elastic search from lambda: see lambda code

In [ ]:
cnt_verify = 0
for index, row in df.iterrows():
    if row["bestAnswer"] == 1 and row["Cuisine"] == "chinese":
        cnt_verify += 1
cnt_verify

In [ ]:
# import boto3
# from botocore.vendored import requests
import json

In [ ]:
headers = {
    'Content-Type': "application/json",
}

In [ ]:
 body = { "sort": [
    {
      "Score": {
        "order": "desc"
      }
    }
  ],
  "query": {
    "term": {
      "Cuisine": "chinese"
    }
  },
  "size": 5
}


In [ ]:
body

In [ ]:
url = "https://search-movies-wyrw3iyi3rnqu3ajemf6skplua.us-east-1.es.amazonaws.com/predictions/Prediction/_search?"

In [ ]:
r = requests.post(url, data = json.dumps(body), headers = headers)

In [ ]:
content = json.loads(r.text)["hits"]["hits"]

In [ ]:
restaurant_ids = []
for i in range(len(content)):
    restaurant_ids.append(content[i]["_source"]["RestaurantID"])

In [ ]:
restaurant_ids

In [ ]:
import json
import requests
headers = {
    'Content-Type': "application/json",
}
body = { "sort": [
    {
      "Score": {
        "order": "desc"
      }
    }
  ],
  "query": {
    "term": {
      "Cuisine": "chinese"
    }
  },
  "size": 5
}
url = "https://search-movies-wyrw3iyi3rnqu3ajemf6skplua.us-east-1.es.amazonaws.com/predictions/Prediction/_search?"
r = requests.post(url, data = json.dumps(body), headers = headers)
content = r.json()["hits"]["hits"]
restaurant_ids = []
for i in range(len(content)):
    restaurant_ids.append(content[i]["_source"]["RestaurantID"])
print(restaurant_ids)